# 시군구코드 한국행정구역분류 변환표 생성

통계 행정구역 분류. 행정동과는 1:1 대응이 되나? 법정동과는 m:n 대응.

In [107]:
from pathlib import Path
import pandas as pd
from tabulate import tabulate

In [108]:
cwd = Path().resolve()
excel_file = (
    cwd.parent / "data/raw/한국행정구역분류_2024.12.31.기준_20241231100033.xlsx"
)
excel_file

WindowsPath('C:/Users/USER/Documents/quacklytics/data/raw/한국행정구역분류_2024.12.31.기준_20241231100033.xlsx')

In [109]:
df = pd.read_excel(
    excel_file,
    sheet_name="2-2. 연계표_행정동 및 법정동(기준시점)",
    header=1,  # Row 2 (0-based index) contains column names
    usecols="A,B,C,D,E,F,G,H,I,J,K",
    dtype=str,
    keep_default_na=False,
    na_values=[
        "#N/A",
        "#NA",
        "<NA>",
        "N/A",
        "NA",
        "NULL",
        "NaN",
        "None",
        "n/a",
        "nan",
        "null",
    ],
)
df.columns = [col.replace("\n", "") for col in df.columns]
df.head()

c:\Users\USER\Documents\quacklytics\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역분할여부,행정동 영문명칭
0,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11,1100000000,19880423,1100000000,,Seoul
1,서울특별시,종로구,종로구,종로구,종로구,11010,1111000000,19880423,1111000000,,Jongno-gu
2,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720,1111051500,20081101,1111010100,,Cheongunhyoja-dong
3,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720,1111051500,20081101,1111010200,,Cheongunhyoja-dong
4,서울특별시,종로구,청운효자동,청운효자동,궁정동,11010720,1111051500,20081101,1111010300,,Cheongunhyoja-dong


법정동코드로 먼저 필터해봄. 법정동코드 뒷자리는 시군구 내 법정동 분류를 의미하고, 그 부분이 00000이면 시군구에 대한 법정동코드가 됨. 마찬가지로, 뒷자리 8자리가 00000000이면 시도에 대한 코드이므로, 이는 제외.

In [110]:
five_zero_df = df[
    (df["법정동코드"].str[-5:] == "00000") & (df["법정동코드"].str[-8:] != "00000000")
]
five_zero_df

,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역분할여부,행정동 영문명칭
1,서울특별시,종로구,종로구,종로구,종로구,11010,1111000000,19880423,1111000000,,Jongno-gu
95,서울특별시,중구,중구,중구,중구,11020,1114000000,19880423,1114000000,,Jung-gu
188,서울특별시,용산구,용산구,용산구,용산구,11030,1117000000,19880423,1117000000,,Yongsan-gu
227,서울특별시,성동구,성동구,성동구,성동구,11040,1120000000,19880423,1120000000,,Seongdong-gu
251,서울특별시,광진구,광진구,광진구,광진구,11050,1121500000,19950301,1121500000,,Gwangjin-gu
...,...,...,...,...,...,...,...,...,...,...,...
21128,경상남도,함양군,함양군,함양군,함양군,38580,4887000000,19880423,4887000000,,Hamyang-gun
21243,경상남도,거창군,거창군,거창군,거창군,38590,4888000000,19880423,4888000000,,Geochang-gun
21350,경상남도,합천군,합천군,합천군,합천군,38600,4889000000,19880423,4889000000,,Hapcheon-gun
21561,제주특별자치도,제주시,제주시,제주시,제주시,39010,5011000000,20060701,5011000000,,Jeju-si


시군구 수(226), 세종시(1), 행정시·자치구가 아닌 구(37), 그 합(264) 등보다 행이 더 많음. 시군구가 아닌 것이 끼어있다는 의미.

In [111]:
five_zero_df["행정구역분류"].str.len().describe()

count    326.000000
mean       4.049080
std        1.965248
min        0.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: 행정구역분류, dtype: float64

In [112]:
five_zero_df[five_zero_df["행정구역분류"].str.len() != 5]

,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역분할여부,행정동 영문명칭
1471,대구광역시,달성군,다사읍서재출장소,다사읍서재출장소,달성군,,2771025700,20010323,2771000000,,-
1709,인천광역시,중구,중구영종출장소,중구영종출장소,중구,,2811400000,20060712,2811400000,,-
1710,인천광역시,중구,중구용유출장소,중구용유출장소,중구,,2811800000,20060712,2811800000,,-
1891,인천광역시,서구,서구검단출장소,서구검단출장소,서구,,2826500000,19950301,2826500000,,-
2002,인천광역시,강화군,서도면볼음출장소,서도면볼음출장소,강화군,,2871042500,19950301,2871000000,,-
...,...,...,...,...,...,...,...,...,...,...,...
19971,경상남도,밀양시,삼랑진임천출장소,삼랑진임천출장소,밀양시,,4827025100,20010126,4827000000,,-
20151,경상남도,거제시,사등면가조출장소,사등면가조출장소,거제시,,4831036600,19950101,4831000000,,-
20172,경상남도,거제시,하청면칠천출장소,하청면칠천출장소,거제시,,4831038500,19950101,4831000000,,-
20184,경상남도,거제시,장목면외포출장소,장목면외포출장소,거제시,,4831039500,19950101,4831000000,,-


출장소 등 62건이 잡힘. 326-62 = 264로 이들만 제거하면 일치함.

이번에는 행정구역명을 기준으로 반대 방향에서 검증해보겠음.

In [113]:
same_b_c_df = df[(df["시도"] != df["행정구역명"]) & (df["시군구"] == df["행정구역명"])]
same_b_c_df

,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역분할여부,행정동 영문명칭
1,서울특별시,종로구,종로구,종로구,종로구,11010,1111000000,19880423,1111000000,,Jongno-gu
95,서울특별시,중구,중구,중구,중구,11020,1114000000,19880423,1114000000,,Jung-gu
188,서울특별시,용산구,용산구,용산구,용산구,11030,1117000000,19880423,1117000000,,Yongsan-gu
227,서울특별시,성동구,성동구,성동구,성동구,11040,1120000000,19880423,1120000000,,Seongdong-gu
251,서울특별시,광진구,광진구,광진구,광진구,11050,1121500000,19950301,1121500000,,Gwangjin-gu
...,...,...,...,...,...,...,...,...,...,...,...
21128,경상남도,함양군,함양군,함양군,함양군,38580,4887000000,19880423,4887000000,,Hamyang-gun
21243,경상남도,거창군,거창군,거창군,거창군,38590,4888000000,19880423,4888000000,,Geochang-gun
21350,경상남도,합천군,합천군,합천군,합천군,38600,4889000000,19880423,4889000000,,Hapcheon-gun
21561,제주특별자치도,제주시,제주시,제주시,제주시,39010,5011000000,20060701,5011000000,,Jeju-si


시군구 수(226), 세종시(1), 행정시·자치구가 아닌 구(37)의 합(264)과 일치하지 않음. 세종시가 빠짐.

In [114]:
final_df = five_zero_df[five_zero_df["행정구역분류"].str.len() == 5]
final_df

,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역분할여부,행정동 영문명칭
1,서울특별시,종로구,종로구,종로구,종로구,11010,1111000000,19880423,1111000000,,Jongno-gu
95,서울특별시,중구,중구,중구,중구,11020,1114000000,19880423,1114000000,,Jung-gu
188,서울특별시,용산구,용산구,용산구,용산구,11030,1117000000,19880423,1117000000,,Yongsan-gu
227,서울특별시,성동구,성동구,성동구,성동구,11040,1120000000,19880423,1120000000,,Seongdong-gu
251,서울특별시,광진구,광진구,광진구,광진구,11050,1121500000,19950301,1121500000,,Gwangjin-gu
...,...,...,...,...,...,...,...,...,...,...,...
21128,경상남도,함양군,함양군,함양군,함양군,38580,4887000000,19880423,4887000000,,Hamyang-gun
21243,경상남도,거창군,거창군,거창군,거창군,38590,4888000000,19880423,4888000000,,Geochang-gun
21350,경상남도,합천군,합천군,합천군,합천군,38600,4889000000,19880423,4889000000,,Hapcheon-gun
21561,제주특별자치도,제주시,제주시,제주시,제주시,39010,5011000000,20060701,5011000000,,Jeju-si


In [115]:
# add a column 시군구코드 as the first 5 characters of 법정동코드
final_df["시군구코드"] = final_df["법정동코드"].str[:5]
# make it index
final_df = final_df.set_index("시군구코드")

final_df.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28996\3015759536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["시군구코드"] = final_df["법정동코드"].str[:5]


,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역분할여부,행정동 영문명칭
시군구코드,,,,,,,,,,,
11110,서울특별시,종로구,종로구,종로구,종로구,11010,1111000000,19880423,1111000000,,Jongno-gu
11140,서울특별시,중구,중구,중구,중구,11020,1114000000,19880423,1114000000,,Jung-gu
11170,서울특별시,용산구,용산구,용산구,용산구,11030,1117000000,19880423,1117000000,,Yongsan-gu
11200,서울특별시,성동구,성동구,성동구,성동구,11040,1120000000,19880423,1120000000,,Seongdong-gu
11215,서울특별시,광진구,광진구,광진구,광진구,11050,1121500000,19950301,1121500000,,Gwangjin-gu


In [119]:
output_dir = cwd.parent / "data/processed"
output_dir.mkdir(parents=True, exist_ok=True)
final_df.to_csv(output_dir / "code_kcad_sgg_2024.csv", index=True, encoding="utf-8-sig")